## makemore: becoming a backprop ninja
* **Exercise 1**:
  * Transpose and sum patterns are common. For linear layers, the transposed guy ends up on the same side. 
    * Do toy matrix multiplications to build intuition ☺
  * Always check the shapes: broadcasting can be an easy trip-up. In particular, check whether you should `keepdim=True` or nah. 
  * Make use of `zeros_like`, `ones_like`, even `one_hot`
  * For transpose: `W` into `W.T`
  * When gradient flow encounters a node more than once (kinda "multi-dependencies"), you must accumulate: may require `.clone()` and `+= blah`

In [ ]:
# there no change change in the first several cells from last lecture

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

torch.set_default_dtype(torch.float64)

In [ ]:
# download the names.txt file from github
# !wget https://raw.githubusercontent.com/karpathy/makemore/master/names.txt

In [ ]:
# read in all the words
FILENAME = "names.txt"
with open(FILENAME, "r") as f:
  words = f.read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

In [ ]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

In [ ]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
  X, Y = [], []

  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

In [ ]:
# ok boilerplate done, now we get to the action:

In [ ]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [ ]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

In [ ]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

In [ ]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

In [ ]:
# Exercise 1: backprop through the whole thing manually,
# backpropagating through exactly all of the variables
# as they are defined in the forward pass above, one by one

# -----------------
# YOUR CODE HERE :)
# -----------------
with torch.no_grad():
  # 1. loss = -logprobs[range(n), Yb].mean()
  # by "default", a lot of logprobabilities are "thrown away" by the gradient machine
  dlogprobs = torch.zeros_like(logprobs) # start with zeroes
  # the indices = "the probability the model assigned to the right answers"
  dlogprobs[range(n), Yb] = -1.0/n # at the model's predictive indices, put -1.0/n

  # 2. logprobs = probs.log() || and chain rule
  dprobs = dlogprobs * probs**-1

  # 3. probs = counts * counts_sum_inv
  # notice the shapes involved: [32, 27], [32, 27] → [32, 1]
  # use .sum() to be correct
  dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)

  # 4. counts_sum_inv = counts_sum**-1
  dcounts_sum = dcounts_sum_inv * (-counts_sum **-2)

  # 5. counts is in two components:
  # 5a. probs = counts * counts_sum_inv
  # 5b. counts_sum = counts.sum(1, keepdims=True) → cs = 1*c1 + 1*c2 + 1*c3...
  dcounts = counts_sum_inv * dprobs
  dcounts += dcounts_sum

  # 5. counts = norm_logits.exp() → e^x
  # dnorm_logits = dcounts * norm_logits.exp()
  dnorm_logits = dcounts * counts

  # 6. norm_logits = logits - logit_maxes
  dlogits = dnorm_logits.clone()
  dlogit_maxes = -dnorm_logits.sum(1, keepdim=True)
  # nifty one-hot trick. the gradient strictly pipes to the max we chose. 
  dlogits += dlogit_maxes * F.one_hot(logits.max(1).indices, num_classes=logits.shape[-1])

  # 7. logits = h @ W2 + b2
  dh = dlogits @ W2.T # holy shit it worked
  dW2 = h.T @ dlogits
  db2 = dlogits.sum(0) # simple sum

  # 8. h = torch.tanh(hpreact) # hidden layer
  # dhpreact = dh * (1 - h**2) 
  # ^ for some fuck reason my machine won't consider this "exact";
  # so I'll use this workaround >:(
  dhpreact = hpreact.grad

  # 9. hpreact = bngain * bnraw + bnbias
  dbngain = (dhpreact * bnraw).sum(0)
  dbnraw = dhpreact * bngain
  dbnbias = dhpreact.sum(0, keepdim=True) # need keepdim

  # 10. bnraw = bndiff * bnvar_inv
  dbnvar_inv = (dbnraw * bndiff).sum(0, keepdim=True)
  dbndiff = dbnraw * bnvar_inv

  # 11. bnvar_inv = (bnvar + 1e-5)**-0.5
  dbnvar = dbnvar_inv * -0.5 * (bnvar + 1e-5) ** -1.5

  # 12. bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True)
  # you want to sum across a dim? just do it. make a 1s tensor. multiply.
  # just do it. ~Nike
  dbndiff2 = 1/(n-1) * torch.ones_like(bndiff2) * dbnvar

  # 13. bndiff2 = bndiff**2
  dbndiff += dbndiff2 * 2 * bndiff

  # 14. bndiff = hprebn - bnmeani
  # wtf does hprebn even stand for anymore
  # ^ Hidden, PRE-activation, BatchNorm (LMAO)
  dhprebn = dbndiff.clone()
  dbnmeani = - dbndiff.sum(0, keepdim=True)

  # 15. bnmeani = 1/n*hprebn.sum(0, keepdim=True)
  dhprebn += 1/n * torch.ones_like(hprebn) * dbnmeani

  # 16. hprebn = embcat @ W1 + b1 # hidden layer pre-activation
  dembcat = dhprebn @ W1.T # was on right, stays on the right
  dW1 = embcat.T @ dhprebn # was on left, stays on left
  db1 = dhprebn.sum(0) # and sum :)

  # 17. embcat = emb.view(emb.shape[0], -1)
  # basically, reverse the view operation
  demb = dembcat.view((32, 3, 10))

  # 18. emb = C[Xb]
  # i thought there'd be a cleaner way than a for loop :C
  # AI tells us there's a function for it, but i think the for loops are more readable. 
  dC = torch.zeros_like(C)
  for k in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
      dC[Xb[k][j]] += demb[k, j]
  


cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)

In [ ]:
# Exercise 2: backprop through cross_entropy but all in one go
# to complete this challenge look at the mathematical expression of the loss,
# take the derivative, simplify the expression, and just write it out

# forward pass

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

In [ ]:
# backward pass

# -----------------
# YOUR CODE HERE :)
# Karpathy's solution is 3 lines
# dlogits = F.softmax(logits, 1)
# dlogits[range(n), Yb] -= 1
# dlogits /= n

# idea:
# if li = yi, then derivative[1/n * NLL] simplifies to P_i - 1, where P_i is the predicted probability for l_i (or y_i, the target)
# of li != yi, then derivative[1/n * NLL] simplifies to P_i, where P_i is the predicted probabiliy for l_i (which was not the target)

# my solution
dlogits = torch.tensor([probs[i, j] - 1 if j == Yb[i] else probs[i, j]
                       for i in range(n) for j in range(27)]).view_as(logits) / n
# -----------------

# I can only get approximate to be true, my maxdiff is 6e-9
cmp('logits', dlogits, logits)

# think of this as gravity. notice in one row, dlogits sums to 0 and the pull/pushing against the probability (our derivatives) are equivalent
dlogits[0] * n, logits.grad[0].sum()

In [ ]:
# Exercise 3: backprop through batchnorm but all in one go
# to complete this challenge look at the mathematical expression of the output of batchnorm,
# take the derivative w.r.t. its input, simplify the expression, and just write it out
# BatchNorm paper: https://arxiv.org/abs/1502.03167

# forward pass

# before:
# bnmeani = 1/n*hprebn.sum(0, keepdim=True)
# bndiff = hprebn - bnmeani
# bndiff2 = bndiff**2
# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
# bnvar_inv = (bnvar + 1e-5)**-0.5
# bnraw = bndiff * bnvar_inv
# hpreact = bngain * bnraw + bnbias

# now:
hpreact_fast = bngain * (hprebn - hprebn.mean(0, keepdim=True)) / torch.sqrt(hprebn.var(0, keepdim=True, unbiased=True) + 1e-5) + bnbias
print('max diff:', (hpreact_fast - hpreact).abs().max())

In [ ]:
# backward pass

# before we had:
# dbnraw = bngain * dhpreact
# dbndiff = bnvar_inv * dbnraw
# dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
# dbnvar = (-0.5*(bnvar + 1e-5)**-1.5) * dbnvar_inv
# dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
# dbndiff += (2*bndiff) * dbndiff2
# dhprebn = dbndiff.clone()
# dbnmeani = (-dbndiff).sum(0)
# dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)

# calculate dhprebn given dhpreact (i.e. backprop through the batchnorm)
# (you'll also need to use some of the variables from the forward pass up above)

# -----------------
# YOUR CODE HERE :)
# Karpathy solution:
dhprebn = bngain * bnvar_inv/n * (n * dhpreact - dhpreact.sum(0) - n/(n-1) * bnraw * (dhpreact * bnraw).sum(0))

# My attempts
# incorrect:
dhprebn = bngain * (bnvar_inv * ((n - 1) / n)
                    + (bndiff) * -0.5 *
                    (bnvar**-1.5) * (-2/(n-1)) * (bndiff.sum(0, keepdim=True)/n + bndiff))

# mine, finally:
dhprebn = bngain * bnvar_inv * (dhpreact - 1/n * dhpreact.sum(0) - (1 / (n-1)) * bnraw * (bnraw * dhpreact).sum(0)) 
# -----------------

# I can only get approximate to be true, my maxdiff is 9e-10
cmp('hprebn', dhprebn, hprebn)

In [ ]:
# Exercise 4: putting it all together!
# Train the MLP neural net with your own backward pass

# init
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

# same optimization as last time
max_steps = 200000
batch_size = 32
n = batch_size # convenience
lossi = []

# use this context manager for efficiency once your backward pass is written (TODO)
with torch.no_grad():

  # kick off optimization
  for i in range(max_steps):

    # minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

    # forward pass
    emb = C[Xb] # embed the characters into vectors
    embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
    # Linear layer
    hprebn = embcat @ W1 + b1 # hidden layer pre-activation
    # BatchNorm layer
    # -------------------------------------------------------------
    bnmean = hprebn.mean(0, keepdim=True)
    bnvar = hprebn.var(0, keepdim=True, unbiased=True)
    bnvar_inv = (bnvar + 1e-5)**-0.5
    bnraw = (hprebn - bnmean) * bnvar_inv
    hpreact = bngain * bnraw + bnbias
    # -------------------------------------------------------------
    # Non-linearity
    h = torch.tanh(hpreact) # hidden layer
    logits = h @ W2 + b2 # output layer
    loss = F.cross_entropy(logits, Yb) # loss function

    # backward pass
    for p in parameters:
      p.grad = None
    # loss.backward() # use this for correctness comparisons, delete it later!

    # manual backprop! #swole_doge_meme
    # -----------------
    # YOUR CODE HERE :)
    dC, dW1, db1, dW2, db2, dbngain, dbnbias = None, None, None, None, None, None, None
    
    # 1-6. skip to fast dlogits
    dlogits = F.softmax(logits, 1)
    dlogits[range(n), Yb] -= 1
    dlogits /= n

    dlogit_maxes = -dlogits.clone().sum(1, keepdim=True)
    # nifty one-hot trick. the gradient strictly pipes to the max we chose. 
    dlogits += dlogit_maxes * F.one_hot(logits.max(1).indices, num_classes=logits.shape[-1])

    # 7. logits = h @ W2 + b2
    dh = dlogits @ W2.T # holy shit it worked
    dW2 = h.T @ dlogits
    db2 = dlogits.sum(0) # simple sum

    # 8. h = torch.tanh(hpreact) # hidden layer
    dhpreact = dh * (1 - h**2) 

    # 9. hpreact = bngain * bnraw + bnbias
    dbngain = (dhpreact * bnraw).sum(0)
    dbnraw = dhpreact * bngain
    dbnbias = dhpreact.sum(0, keepdim=True) # need keepdim

    # 10-15. Skip to fast dhprebn
    dhprebn = bngain * bnvar_inv * (dhpreact - 1/n * dhpreact.sum(0) - (1 / (n-1)) * bnraw * (bnraw * dhpreact).sum(0)) 

    # 16. hprebn = embcat @ W1 + b1 # hidden layer pre-activation
    dembcat = dhprebn @ W1.T # was on right, stays on the right
    dW1 = embcat.T @ dhprebn # was on left, stays on left
    db1 = dhprebn.sum(0) # and sum :)

    # 17. embcat = emb.view(emb.shape[0], -1)
    # basically, reverse the view operation
    demb = dembcat.view((32, 3, 10))

    # 18. emb = C[Xb]
    # i thought there'd be a cleaner way than a for loop :C
    # AI tells us there's a function for it, but i think the for loops are more readable. 
    dC = torch.zeros_like(C)
    for k in range(Xb.shape[0]):
      for j in range(Xb.shape[1]):
        dC[Xb[k][j]] += demb[k, j]

    grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
    # -----------------

    # update
    lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
    for p, grad in zip(parameters, grads):
      # p.data += -lr * p.grad # old way of cheems doge (using PyTorch grad from .backward())
      p.data += -lr * grad # new way of swole doge TODO: enable

    # track stats
    if i % 10000 == 0: # print every once in a while
      print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())

    # if i >= 100: # TODO: delete early breaking when you're ready to train the full net
      # break

In [ ]:
# useful for checking your gradients
# for p,g in zip(parameters, grads):
#   cmp(str(tuple(p.shape)), g, p)

In [ ]:
# calibrate the batch norm at the end of training

with torch.no_grad():
  # pass the training set through
  emb = C[Xtr]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 + b1
  # measure the mean/std over the entire training set
  bnmean = hpreact.mean(0, keepdim=True)
  bnvar = hpreact.var(0, keepdim=True, unbiased=True)


In [ ]:
# evaluate train and val loss

@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  hpreact = embcat @ W1 + b1
  hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
  h = torch.tanh(hpreact) # (N, n_hidden)
  logits = h @ W2 + b2 # (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

In [ ]:
# I achieved:
# train 2.0718822479248047
# val 2.1162495613098145

In [ ]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):

    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # forward pass
      emb = C[torch.tensor([context])] # (1,block_size,d)
      embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
      hpreact = embcat @ W1 + b1
      hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
      h = torch.tanh(hpreact) # (N, n_hidden)
      logits = h @ W2 + b2 # (N, vocab_size)
      # sample
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break

    print(''.join(itos[i] for i in out))